In [2]:
import sqlite3
from tkinter import Tk, Label, Entry, Button, Listbox, Scrollbar, StringVar, END
from datetime import datetime
import os

# Database setup
def initialize_db():
    # Lấy đường dẫn gốc của thư mục làm việc hiện tại
    current_working_directory = os.getcwd()
    
    conn = sqlite3.connect(os.path.join(current_working_directory, "sales_management.db"))
    cursor = conn.cursor()
    # cursor.execute('''CREATE TABLE IF NOT EXISTS MatHang (
    #     id VARCHAR(10) PRIMARY KEY,
    #     ten_hang VARCHAR(255),
    #     don_vi VARCHAR(20),
    #     so_luong INTEGER,
    #     gia_le INTEGER,
    #     gia_si INTEGER,
    #     ngay_tao DATE DEFAULT CURRENT_DATE,
    #     is_active BOOLEAN DEFAULT 1
    # )''')
    # cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si)
    #                   VALUES ('MH001', 'Bánh mì', 'Cái', 100, 5000, 4500)''')
    # cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si)
    #                   VALUES ('MH002', 'Coca Cola', 'Lon', 50, 10000, 9000)''')
    # cursor.execute('''
    #                CREATE INDEX IF NOT EXISTS idx_id ON MatHang (id)
    #                ''')
    conn.commit()
    conn.close()

# GUI setup
class SalesManagementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sales Management - MatHang")

        # Input fields
        Label(root, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_var = StringVar()
        Entry(root, textvariable=self.id_var).grid(row=0, column=1, padx=5, pady=5)

        Label(root, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
        self.ten_hang_var = StringVar()
        Entry(root, textvariable=self.ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

        Label(root, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
        self.don_vi_var = StringVar()
        Entry(root, textvariable=self.don_vi_var).grid(row=2, column=1, padx=5, pady=5)

        Label(root, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
        self.so_luong_var = StringVar()
        Entry(root, textvariable=self.so_luong_var).grid(row=3, column=1, padx=5, pady=5)

        Label(root, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
        self.gia_le_var = StringVar()
        Entry(root, textvariable=self.gia_le_var).grid(row=4, column=1, padx=5, pady=5)

        Label(root, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
        self.gia_si_var = StringVar()
        Entry(root, textvariable=self.gia_si_var).grid(row=5, column=1, padx=5, pady=5)

        # Buttons
        Button(root, text="Thêm", command=self.create_item).grid(row=6, column=0, padx=5, pady=5)
        Button(root, text="Sửa", command=self.update_item).grid(row=6, column=1, padx=5, pady=5)
        Button(root, text="Xóa", command=self.delete_item).grid(row=6, column=2, padx=5, pady=5)
        Button(root, text="Làm mới", command=self.refresh_list).grid(row=6, column=3, padx=5, pady=5)

        # Listbox
        self.listbox = Listbox(root, width=100, height=20)
        self.listbox.grid(row=7, column=0, columnspan=4, padx=5, pady=5)

        # Scrollbar
        scrollbar = Scrollbar(root, orient="vertical", command=self.listbox.yview)
        scrollbar.grid(row=7, column=4, sticky="ns")
        self.listbox.configure(yscrollcommand=scrollbar.set)

        self.refresh_list()

    # CRUD operations
    def create_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        ngay_tao = datetime.now().strftime("%Y-%m-%d")
        is_active = 1

        try:
            cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si, ngay_tao, is_active)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                           (self.id_var.get(), self.ten_hang_var.get(), self.don_vi_var.get(), int(self.so_luong_var.get()),
                            int(self.gia_le_var.get()), int(self.gia_si_var.get()), ngay_tao, is_active))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def update_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute('''UPDATE MatHang
                              SET ten_hang = ?, don_vi = ?, so_luong = ?, gia_le = ?, gia_si = ?, is_active = ?
                              WHERE id = ?''',
                           (self.ten_hang_var.get(), self.don_vi_var.get(), int(self.so_luong_var.get()),
                            int(self.gia_le_var.get()), int(self.gia_si_var.get()), 1, self.id_var.get()))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def delete_item(self):
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute("DELETE FROM MatHang WHERE id = ?", (self.id_var.get(),))
            conn.commit()
            self.refresh_list()
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def refresh_list(self):
        self.listbox.delete(0, END)
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()

        try:
            cursor.execute("SELECT * FROM MatHang ORDER BY ten_hang")
            rows = cursor.fetchall()
            for row in rows:
                self.listbox.insert(END, row)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            conn.close()

# Run application
if __name__ == "__main__":
    # initialize_db()
    root = Tk()
    app = SalesManagementApp(root)
    root.mainloop()


In [ ]:
from tkinter import *
from static.css.ButtonType import ButtonType
from static.css.LabelType import LabelType

# Tạo cửa sổ chính
root = Tk()
root.geometry("1920x1080")
root.title("Sales Management - MatHang")

# Tạo các Frame (container) cho các giao diện khác nhau
frame_main = Frame(root, bg="lightblue", width=1920, height=1080)
frame_mat_hang = Frame(root, bg="lightgreen", width=1920, height=1080)
frame_nhap_hang = Frame(root, bg="lightyellow", width=1920, height=1080)

# Sử dụng pack để đặt các Frame trong cửa sổ chính
for frame in (frame_main, frame_mat_hang, frame_nhap_hang):
    frame.grid(row=0, column=0, sticky='nsew')

# Hàm chuyển đổi giữa các Frame
def show_frame(frame):
    frame.tkraise()

# Tạo menu chính
menu_bar = Menu(root)

# Tạo menu Điều hướng
nav_menu = Menu(menu_bar, tearoff=0)
nav_menu.add_command(label="Mặt hàng", command=lambda: show_frame(frame_mat_hang))
nav_menu.add_command(label="Nhập hàng", command=lambda: show_frame(frame_nhap_hang))
menu_bar.add_cascade(label="Điều hướng", menu=nav_menu)

# Gắn menu vào cửa sổ chính
root.config(menu=menu_bar)

# Nội dung của frame_main
label_main = Label(frame_main, text="Chào mừng đến với hệ thống quản lý bán hàng", font=("Arial", 24))
label_main.pack(pady=20)

# Nội dung của frame_mat_hang
label_mat_hang = Label(frame_mat_hang, text="Quản lý mặt hàng", font=("Arial", 24))
label_mat_hang.pack(pady=20)

# Nội dung của frame_nhap_hang
label_nhap_hang = Label(frame_nhap_hang, text="Quản lý nhập hàng", font=("Arial", 24))
label_nhap_hang.pack(pady=20)

# Hiển thị frame_main khi khởi động
show_frame(frame_main)

# Chạy vòng lặp chính của Tkinter
root.mainloop()

In [9]:
# 1111
from tkinter import *
from static.css.ButtonType import ButtonType
from static.css.LabelType import LabelType

# Tạo cửa sổ chính
root = Tk()
# root.attributes("-fullscreen", True)  # Hiển thị cửa sổ ở chế độ toàn màn hình
root.geometry("1920x1080")
root.title("Sales Management - MatHang")

# Tạo các Frame (container) cho các giao diện khác nhau
frame_main = Frame(root, bg="lightblue")
frame_mat_hang = Frame(root, bg="lightgreen")
frame_nhap_hang = Frame(root, bg="lightyellow")

# Sử dụng grid để đặt các Frame trong cửa sổ chính
for frame in (frame_main, frame_mat_hang, frame_nhap_hang):
    frame.grid(row=0, column=0, sticky='nsew')

# Đặt trọng số cho các hàng và cột để các phần tử thay đổi kích thước theo cửa sổ
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)

# Hàm chuyển đổi giữa các Frame
def show_frame(frame):
    frame.tkraise()

# Tạo menu chính
menu_bar = Menu(root)

# Tạo menu Điều hướng
nav_menu = Menu(menu_bar, tearoff=0)
nav_menu.add_command(label="Mặt hàng", command=lambda: show_frame(frame_mat_hang))
nav_menu.add_command(label="Nhập hàng", command=lambda: show_frame(frame_nhap_hang))
menu_bar.add_cascade(label="Điều hướng", menu=nav_menu)

# Gắn menu vào cửa sổ chính
root.config(menu=menu_bar)

# Nội dung của frame_main
label_main = Label(frame_main, text="Chào mừng đến với hệ thống quản lý bán hàng", font=("Arial", 24))
label_main.grid(row=0, column=0, pady=20)

# Nội dung của frame_mat_hang
label_mat_hang = Label(frame_mat_hang, text="Quản lý mặt hàng", font=("Arial", 24))
label_mat_hang.grid(row=0, column=0, pady=20)

# Tạo các Frame con trong frame_mat_hang
sub_frame1 = Frame(frame_mat_hang, bg="lightblue", bd=2, relief="sunken")
sub_frame2 = Frame(frame_mat_hang, bg="lightyellow", bd=2, relief="sunken")

# Sử dụng grid để đặt các Frame con trong frame_mat_hang
sub_frame1.grid(row=1, column=0, padx=10, pady=10, sticky="nsew")
sub_frame2.grid(row=2, column=0, padx=10, pady=10, sticky="nsew")

# Đặt trọng số cho các hàng và cột của frame_mat_hang để các Frame con thay đổi kích thước theo frame_mat_hang
frame_mat_hang.grid_rowconfigure(1, weight=1)
frame_mat_hang.grid_rowconfigure(2, weight=2)
frame_mat_hang.grid_columnconfigure(0, weight=1)
frame_mat_hang.grid_columnconfigure(1, weight=1)

# Thêm nội dung vào sub_frame1
label_sub1 = Label(sub_frame1, text="Sub Frame 1", font=("Arial", 16))
label_sub1.pack(pady=20)

# Thêm nội dung vào sub_frame2
# label_sub2 = Label(sub_frame2, text="Sub Frame 2", font=("Arial", 16))
# label_sub2.grid(row=0, column=0, pady=20)
# Thêm các Label và Button vào scrollable_frame
for i in range(50):
    for j in range(10):
        label = Label(sub_frame2, text=f"Row {i}, Column {j}")
        if i % 2 == 0:
            label.config(bg="lightblue")
        if j % 2 == 0:
            label.config(fg="red")

        label.grid(row=i, column=j, padx=5, pady=5)

# Nội dung của frame_nhap_hang
label_nhap_hang = Label(frame_nhap_hang, text="Quản lý nhập hàng", font=("Arial", 24))
label_nhap_hang.grid(row=0, column=0, pady=20)

# Hiển thị frame_main khi khởi động
show_frame(frame_main)

# Chạy vòng lặp chính của Tkinter
root.mainloop()

In [8]:
import tkinter as tk
from tkinter import Menu
from tkinter import font

# def create_logo(root):
#     # Tải hình ảnh logo
#     image = Image.open("path/to/logo.png")  # Đường dẫn đến file logo
#     photo = ImageTk.PhotoImage(image)

#     # Tạo Label để hiển thị logo
#     logo_label = tk.Label(root, image=photo)
#     logo_label.image = photo  # Giữ tham chiếu đến hình ảnh để tránh bị xóa bởi garbage collector
#     logo_label.grid(row=0, column=0, padx=10, pady=10)

def create_menu(root):
    # Tạo menu chính
    menu_bar = Menu(root, font=custom_font)

    # Tạo menu File
    file_menu = Menu(menu_bar, tearoff=0, font=custom_font)
    file_menu.add_command(label="New", font=custom_font)
    file_menu.add_command(label="Open", font=custom_font)
    file_menu.add_command(label="Save", font=custom_font)
    file_menu.add_separator()
    file_menu.add_command(label="Exit", command=root.quit, font=custom_font)
    menu_bar.add_cascade(label="File", menu=file_menu)

    # Tạo menu Edit
    edit_menu = Menu(menu_bar, tearoff=0, font=custom_font)
    edit_menu.add_command(label="Undo", font=custom_font)
    edit_menu.add_command(label="Redo", font=custom_font)
    menu_bar.add_cascade(label="Edit", menu=edit_menu)

    # Tạo menu Help
    help_menu = Menu(menu_bar, tearoff=0, font=custom_font)
    help_menu.add_command(label="About", font=custom_font)
    menu_bar.add_cascade(label="Help", menu=help_menu)

    # Gắn menu vào cửa sổ chính
    root.config(menu=menu_bar)
    
# Tạo cửa sổ chính
root = tk.Tk()

# Thiết lập chiều dài và rộng của màn hình
width = 1920
height = 1080
root.geometry(f"{width}x{height}")

# Thiết lập tiêu đề cho cửa sổ
root.title("Sales Management - MatHang")

# Tạo phông chữ tùy chỉnh
custom_font = font.Font(family="Arial", size=11, weight="bold")

# Tạo logo và menu
# create_logo(root)
create_menu(root)

# Chạy vòng lặp chính của Tkinter
root.mainloop()

In [13]:
import tkinter as tk
from tkinter import Menu, messagebox, Toplevel, Label, Entry, Button, Scrollbar, StringVar, font
from tkinter import ttk
# from PIL import Image, ImageTk
import sqlite3



# # Hàm tạo logo
# def create_logo(root):
#     # Tải hình ảnh logo
#     image = Image.open("path/to/logo.png")  # Đường dẫn đến file logo
#     photo = ImageTk.PhotoImage(image)

#     # Tạo Label để hiển thị logo
#     logo_label = tk.Label(root, image=photo)
#     logo_label.image = photo  # Giữ tham chiếu đến hình ảnh để tránh bị xóa bởi garbage collector
#     logo_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Hàm tạo menu
def create_menu(root):
    # Tạo menu chính
    menu_bar = Menu(root, font=custom_font)

    # Tạo menu File
    file_menu = Menu(menu_bar, tearoff=0, font=custom_font)
    file_menu.add_command(label="Mặt hàng", font=custom_font)
    file_menu.add_command(label="Nhập hàng", font=custom_font)
    file_menu.add_command(label="Bán hàng", font=custom_font)
    file_menu.add_command(label="Chi phí", font=custom_font)
    file_menu.add_command(label="Khách hàng", font=custom_font)
    file_menu.add_command(label="NCC", font=custom_font)
    file_menu.add_command(label="Báo cáo", font=custom_font)
    menu_bar.add_cascade(label="Điều hướng", menu=file_menu)

    # Gắn menu vào cửa sổ chính
    root.config(menu=menu_bar)

# Hàm thêm mặt hàng
def add_item():
    def save_item():
        # Lưu mặt hàng vào cơ sở dữ liệu
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO MatHang (id, ten_hang, don_vi, so_luong, gia_le, gia_si, ngay_tao, is_active)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                       (id_var.get(), ten_hang_var.get(), don_vi_var.get(), int(so_luong_var.get()),
                        int(gia_le_var.get()), int(gia_si_var.get()), ngay_tao_var.get(), 1))
        conn.commit()
        conn.close()
        add_item_window.destroy()
        refresh_list()

    add_item_window = Toplevel(root)
    add_item_window.title("Thêm mặt hàng")

    Label(add_item_window, text="ID:").grid(row=0, column=0, padx=5, pady=5)
    id_var = StringVar()
    Entry(add_item_window, textvariable=id_var).grid(row=0, column=1, padx=5, pady=5)

    Label(add_item_window, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
    ten_hang_var = StringVar()
    Entry(add_item_window, textvariable=ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

    Label(add_item_window, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
    don_vi_var = StringVar()
    Entry(add_item_window, textvariable=don_vi_var).grid(row=2, column=1, padx=5, pady=5)

    Label(add_item_window, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
    so_luong_var = StringVar()
    Entry(add_item_window, textvariable=so_luong_var).grid(row=3, column=1, padx=5, pady=5)

    Label(add_item_window, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
    gia_le_var = StringVar()
    Entry(add_item_window, textvariable=gia_le_var).grid(row=4, column=1, padx=5, pady=5)

    Label(add_item_window, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
    gia_si_var = StringVar()
    Entry(add_item_window, textvariable=gia_si_var).grid(row=5, column=1, padx=5, pady=5)

    Label(add_item_window, text="Ngày tạo:").grid(row=6, column=0, padx=5, pady=5)
    ngay_tao_var = StringVar()
    Entry(add_item_window, textvariable=ngay_tao_var).grid(row=6, column=1, padx=5, pady=5)

    Button(add_item_window, text="Lưu", command=save_item).grid(row=7, column=0, columnspan=2, padx=5, pady=5)

# Hàm làm mới danh sách mặt hàng
def refresh_list():
    for item in tree.get_children():
        tree.delete(item)
    conn = sqlite3.connect("sales_management.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MatHang ORDER BY ten_hang")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", tk.END, values=row)
    conn.close()

# Hàm xem chi tiết/sửa mặt hàng
def view_edit_item(item_id):
    def save_changes():
        # Lưu thay đổi vào cơ sở dữ liệu
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute('''UPDATE MatHang
                          SET ten_hang = ?, don_vi = ?, so_luong = ?, gia_le = ?, gia_si = ?, is_active = ?
                          WHERE id = ?''',
                       (ten_hang_var.get(), don_vi_var.get(), int(so_luong_var.get()),
                        int(gia_le_var.get()), int(gia_si_var.get()), 1, item_id))
        conn.commit()
        conn.close()
        view_edit_window.destroy()
        refresh_list()

    view_edit_window = Toplevel(root)
    view_edit_window.title("Xem chi tiết/Sửa mặt hàng")

    conn = sqlite3.connect("sales_management.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MatHang WHERE id = ?", (item_id,))
    item = cursor.fetchone()
    conn.close()

    Label(view_edit_window, text="ID:").grid(row=0, column=0, padx=5, pady=5)
    Label(view_edit_window, text=item[0]).grid(row=0, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Tên hàng:").grid(row=1, column=0, padx=5, pady=5)
    ten_hang_var = StringVar(value=item[1])
    Entry(view_edit_window, textvariable=ten_hang_var).grid(row=1, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Đơn vị:").grid(row=2, column=0, padx=5, pady=5)
    don_vi_var = StringVar(value=item[2])
    Entry(view_edit_window, textvariable=don_vi_var).grid(row=2, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Số lượng:").grid(row=3, column=0, padx=5, pady=5)
    so_luong_var = StringVar(value=item[3])
    Entry(view_edit_window, textvariable=so_luong_var).grid(row=3, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Giá lẻ:").grid(row=4, column=0, padx=5, pady=5)
    gia_le_var = StringVar(value=item[4])
    Entry(view_edit_window, textvariable=gia_le_var).grid(row=4, column=1, padx=5, pady=5)

    Label(view_edit_window, text="Giá sỉ:").grid(row=5, column=0, padx=5, pady=5)
    gia_si_var = StringVar(value=item[5])
    Entry(view_edit_window, textvariable=gia_si_var).grid(row=5, column=1, padx=5, pady=5)

    Button(view_edit_window, text="Lưu", command=save_changes).grid(row=6, column=0, columnspan=2, padx=5, pady=5)

# Hàm xóa mặt hàng
def delete_item(item_id, item_name):
    def confirm_delete():
        conn = sqlite3.connect("sales_management.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM MatHang WHERE id = ?", (item_id,))
        conn.commit()
        conn.close()
        delete_window.destroy()
        refresh_list()

    delete_window = Toplevel(root)
    delete_window.title("Xóa mặt hàng")

    Label(delete_window, text=f"Bạn có muốn xóa {item_name} này không?").grid(row=0, column=0, padx=5, pady=5)
    Button(delete_window, text="Có", command=confirm_delete).grid(row=1, column=0, padx=5, pady=5)
    Button(delete_window, text="Không", command=delete_window.destroy).grid(row=1, column=1, padx=5, pady=5)

# Tạo cửa sổ chính
root = tk.Tk()
root.geometry("800x600")
root.title("Sales Management - MatHang")
# Tạo phông chữ tùy chỉnh
custom_font = font.Font(family="Helvetica", size=12, weight="bold")
# Tạo logo và menu
# create_logo(root)
create_menu(root)

# Tiêu đề
Label(root, text="Quản lý mặt hàng", font=custom_font).grid(row=1, column=0, columnspan=4, padx=10, pady=10)

# Thông tin tổng mặt hàng và tổng số lượng hàng
total_items_label = Label(root, text="Tổng mặt hàng: 0", font=custom_font)
total_items_label.grid(row=2, column=2, padx=10, pady=10, sticky="e")

total_quantity_label = Label(root, text="Tổng số lượng hàng: 0", font=custom_font)
total_quantity_label.grid(row=2, column=3, padx=10, pady=10, sticky="e")

# Nút thêm mặt hàng
Button(root, text="Thêm mặt hàng", command=add_item, font=custom_font).grid(row=2, column=0, padx=10, pady=10, sticky="w")

# Bảng hiển thị danh sách mặt hàng
columns = ("id", "ten_hang", "don_vi", "so_luong", "gia_le", "gia_si", "ngay_tao", "is_active", "actions")
tree = ttk.Treeview(root, columns=columns, show="headings")
tree.heading("id", text="ID")
tree.heading("ten_hang", text="Tên hàng")
tree.heading("don_vi", text="Đơn vị")
tree.heading("so_luong", text="Số lượng")
tree.heading("gia_le", text="Giá lẻ")
tree.heading("gia_si", text="Giá sỉ")
tree.heading("ngay_tao", text="Ngày tạo")
tree.heading("is_active", text="Hoạt động")
tree.heading("actions", text="Hành động")

tree.column("actions", width=200)

tree.grid(row=3, column=0, columnspan=4, padx=10, pady=10)

# Scrollbar
scrollbar = Scrollbar(root, orient="vertical", command=tree.yview)
scrollbar.grid(row=3, column=4, sticky="ns")
tree.configure(yscrollcommand=scrollbar.set)

# Làm mới danh sách mặt hàng
refresh_list()

# Chạy vòng lặp chính của Tkinter
root.mainloop()

In [10]:
import sqlite3
import os
from datetime import datetime

class MatHang:
    def __init__(self, id, ten_hang, don_vi, so_luong, gia_le, gia_si, ngay_tao, is_active):
        self.id = id
        self.ten_hang = ten_hang
        self.don_vi = don_vi
        self.so_luong = so_luong
        self.gia_le = gia_le
        self.gia_si = gia_si
        self.ngay_tao = ngay_tao if ngay_tao is not None else datetime.now().strftime("%Y-%m-%d")
        self.is_active = is_active

    def __repr__(self):
        return f"MatHang(id={self.id}, ten_hang={self.ten_hang}, don_vi={self.don_vi}, so_luong={self.so_luong}, gia_le={self.gia_le}, gia_si={self.gia_si}, ngay_tao={self.ngay_tao}, is_active={self.is_active})"
    def to_tuple(self):
        return (self.id, self.ten_hang, self.don_vi, self.so_luong, self.gia_le, self.gia_si, self.ngay_tao, self.is_active)

# Database setup
def initialize_db():
    # Lấy đường dẫn gốc của thư mục làm việc hiện tại
    current_working_directory = os.getcwd()
    conn = sqlite3.connect(os.path.join(current_working_directory, "sales_management.db"))
    cursor = conn.cursor()
    mat_hang = MatHang('MH007', 'Pepsi', 'Lon', 30, 12000, 11000, None, 1)
    # cursor.execute('''UPDATE MatHang SET is_active = 0 WHERE id = 'MH001' ''')
    # conn.commit()
    cursor.execute('INSERT INTO MatHang VALUES (?, ?, ?, ?, ?, ?, ?, ?)', mat_hang.to_tuple())
    conn.commit()
    cursor.execute('''SELECT * FROM MatHang''')
    data = cursor.fetchall()
    print(data)
    print(type(data))
    print(len(data))
    print(type(data[0]))
    # Chuyển đổi các hàng dữ liệu thành các đối tượng MatHang
    mat_hang_list = [MatHang(*row) for row in data]
    
    # In ra danh sách các đối tượng MatHang
    for mat_hang in mat_hang_list:
        print(mat_hang)
    conn.close()

if __name__ == "__main__":
    initialize_db()

[('MH001', 'Bánh mì', 'Cái', 100, 5000, 4500, '2025-01-13', 0), ('MH002', 'Coca Cola', 'Lon', 50, 10000, 9000, '2025-01-13', 1), ('MH003', 'Pepsi', 'Lon', 30, 12000, 11000, '2025-01-13', 1), ('MH004', 'Pepsi', 'Lon', 30, 12000, 11000, '2021-07-01', 1), ('MH007', 'Pepsi', 'Lon', 30, 12000, 11000, '2025-01-13', 1)]
<class 'list'>
5
<class 'tuple'>
MatHang(id=MH001, ten_hang=Bánh mì, don_vi=Cái, so_luong=100, gia_le=5000, gia_si=4500, ngay_tao=2025-01-13, is_active=0)
MatHang(id=MH002, ten_hang=Coca Cola, don_vi=Lon, so_luong=50, gia_le=10000, gia_si=9000, ngay_tao=2025-01-13, is_active=1)
MatHang(id=MH003, ten_hang=Pepsi, don_vi=Lon, so_luong=30, gia_le=12000, gia_si=11000, ngay_tao=2025-01-13, is_active=1)
MatHang(id=MH004, ten_hang=Pepsi, don_vi=Lon, so_luong=30, gia_le=12000, gia_si=11000, ngay_tao=2021-07-01, is_active=1)
MatHang(id=MH007, ten_hang=Pepsi, don_vi=Lon, so_luong=30, gia_le=12000, gia_si=11000, ngay_tao=2025-01-13, is_active=1)


Đường dẫn gốc của thư mục làm việc hiện tại là: d:\documents\PythonProjects\SalesManagementApp
